In [1]:

import qsharp
from qsharp import azure
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'qsharp'

In [ ]:
%%qsharp 
open Microsoft.Quantum.Diagnostics; 
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Random;

In [ ]:
%%qsharp
/// # Summary
/// A phase oracle which applies a negative phase to 
/// the states where no adjacent bits are equal.
///
/// # Input 
/// ## qs 
/// The qubits the oracle is applied to. 

operation AlternatingBitStringOracle(qs : Qubit[]) : Unit { 
    // We first apply a CNOT gate between each pair of adjacent qubits
    // to calculate qs[i] XOR qs[i-1] and store the result in qs[i-1].
    // Next, we apply a CZ to the output of all of these XOR operations
    // flipping the phase of the state where each XOR evaluates to 1.
    // Then we uncompute the previous CNOT gates to return the phase
    // to the target states. 

    // The within-apply statement does 3 things:
    // 1. Performs the operations in the within block.
    // 2. Performs the operations in the apply block.
    // 3. Calculates and performs the adjoint of the within block.
    Controlled Z(Most(qs), Tail(qs));
}

In [ ]:
%%qsharp
/// # Summary
/// Grover's diffusion operator.
///
/// # Input
/// ## qs
/// The qubit array the diffusion operator is applied to.

operation Diffusion(qs : Qubit[]) : Unit {
    within {
        ApplyToEachA(H, qs);
        ApplyToEachA(X, qs);
    } apply {
        // Most gives us every array element except the last.
        // Tail gives us the last array element.
        Controlled Z(Most(qs), Tail(qs));
    }
}

In [ ]:
%%qsharp
/// # Summary
/// The top-level operation for this implementation. 
/// Applies the oracle and the diffusion operator multiple times.
///
/// # Input
/// ## iterations
/// The number of times to apply the oracle and the diffusion operator.
/// ## N 
/// The number of bits in the bit strings. Our search space size is then 2^N.
///
/// # Output
/// A Result[] type which contains the measurement result of each
/// qubit that was allocated. 

operation Grovers(iterations : Int, N : Int) : Result[] {
    use qs = Qubit[N];
    ApplyToEach(H, qs);

    for i in 1 .. iterations {
        AlternatingBitStringOracle(qs);
        Diffusion(qs);
    }
    return ForEach(M, qs);
}

In [ ]:
Grovers.simulate(iterations=1, N=3)

In [ ]:
qsharp.azure.connect(
   resourceId="",
   location="")

In [ ]:

qsharp.azure.target("ionq.qpu")

In [ ]:
qsharp.azure.target("ionq.qpu")
N = 3
iterations = 1
qsharp.azure.submit(Grovers, iterations=iterations, N=N, shots=500, jobName="Grover's iterations={}, N={}".format(iterations, N))

In [ ]:
qsharp.azure.status()

In [ ]:
def plot_job_results(id, iter, sim):
    output = qsharp.azure.output(id)
    print(output)
    keys = list(output.keys())
    keys = [key.replace(",", "") for key in keys]
    keyLen = len(keys[0]) - 2
    firstBitString = ""
    secondBitString = ""
    for i in range(keyLen):
        firstBitString += str((i+1)%2)
        secondBitString += str(i%2)
    secondBitString = "[{}]".format(secondBitString)
    firstBitString = "[{}]".format(firstBitString)
    outputFreq = list(output.values())

    firstBitStringLoc = min(keys.index(firstBitString), keys.index(secondBitString))
    secondBitStringLoc = max(keys.index(firstBitString), keys.index(secondBitString))

    plt.bar(keys[0:firstBitStringLoc], outputFreq[0:firstBitStringLoc], color = "blue")
    plt.bar(keys[firstBitStringLoc], outputFreq[firstBitStringLoc], color = "red")
    plt.bar(keys[firstBitStringLoc+1:secondBitStringLoc], outputFreq[firstBitStringLoc+1:secondBitStringLoc], color = "blue")
    plt.bar(keys[secondBitStringLoc], outputFreq[secondBitStringLoc], color = "red")
    plt.bar(keys[secondBitStringLoc+1:], outputFreq[secondBitStringLoc+1:], color = "blue")
    plt.xticks(rotation=90)
    plt.title("{} results for N = {} iter = {}".format(sim, keyLen, iter))
    plt.show()

In [ ]:
plot_job_results("", iterations, "(target name)")